# Code Generation & Conversion

GPT4 is expert in code translations.

In [14]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential, path="workspace.json")
except Exception as ex:
    raise Exception(
        "Failed to create MLClient from config file. Please modify and then run the above cell with your AzureML Workspace details."
    ) from ex
    
ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ml_client)

Found the config file in: workspace.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f4588b24eb0>,
         subscription_id=fe38c376-b42a-4741-9e7c-f5d7c31e5873,
         resource_group_name=yelizkilinc-rg,
         workspace_name=aml-prod)


### Setup Parameters

In [15]:
keyvault = ws.get_default_keyvault()

aoai_endpoint=keyvault.get_secret(name="aoai-endpoint")
aoai_key=keyvault.get_secret(name="key")

In [16]:
# Set up Azure OpenAI
import openai

openai.api_type = "azure"
openai.api_base = aoai_endpoint # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
openai.api_version = "2023-03-15-preview"
openai.api_key = aoai_key

In [17]:
# Defining a function to send the prompt to the AOAI model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500):
    response = openai.ChatCompletion.create(
        engine=model_name,
        messages=messages,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

In [24]:
base_system_message = "You are a helpful assistant that converts SAS code into Python."

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant that converts SAS code into Python.


In [25]:
# This is the first user message that will be sent to the model. Feel free to update this.
user_message = f"""data work.experience;
input employee $ experience;
datalines;
Ahmet 10
Fatma 8
Ali 12
Elif 7
John 4
Jade 15
;
run;
"""

In [26]:
# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [27]:
# A sample API call for chat completions looks as follows:
# Messages must be an array of message objects, where each object has a role (either "system", "user", or "assistant") and content (the content of the message).
# For more info: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#chat-completions
# You can get "resource not found error in case your version isn't correct"

model_name= "gpt-4-32k"

try:
    max_response_tokens = 500

    response = send_message(messages, model_name, max_response_tokens)
    messages.append({"role": "assistant", "content": response})

    print_conversation(messages)    
   
except openai.error.APIError as e:
    # Handle API error here, e.g. retry or log
    print(f"OpenAI API returned an API Error: {e}")

except openai.error.AuthenticationError as e:
    # Handle Authentication error here, e.g. invalid API key
    print(f"OpenAI API returned an Authentication Error: {e}")

except openai.error.APIConnectionError as e:
    # Handle connection error here
    print(f"Failed to connect to OpenAI API: {e}")

except openai.error.InvalidRequestError as e:
    # Handle connection error here
    print(f"Invalid Request Error: {e}")

except openai.error.RateLimitError as e:
    # Handle rate limit error
    print(f"OpenAI API request exceeded rate limit: {e}")

except openai.error.ServiceUnavailableError as e:
    # Handle Service Unavailable error
    print(f"Service Unavailable: {e}")

except openai.error.Timeout as e:
    # Handle request timeout
    print(f"Request timed out: {e}")
    
except:
    # Handles all other exceptions
    print("An exception has occured.")

[SYSTEM]
You are a helpful assistant that converts SAS code into Python.

[USER]
data work.experience;
input employee $ experience;
datalines;
Ahmet 10
Fatma 8
Ali 12
Elif 7
John 4
Jade 15
;
run;


[ASSISTANT]
{
  "id": "chatcmpl-7oAr8S2lHhcmNMaFpFUAjbDgPGeDJ",
  "object": "chat.completion",
  "created": 1692192486,
  "model": "gpt-4-32k",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "In Python, you can use pandas DataFrame to handle this task. Here is the equivalent code:\n\n```python\nimport pandas as pd\n\n# Create a dictionary\ndata = {'employee': ['Ahmet', 'Fatma', 'Ali', 'Elif', 'John', 'Jade'],\n        'experience': [10, 8, 12, 7, 4, 15]}\n\n# Convert the dictionary into DataFrame \ndf = pd.DataFrame(data)\n\n# Print the data\nprint(df)\n```\n\nThis will create a DataFrame with the same data as in your SAS code. The 'employee' and 'experience' are columns in the DataFrame."
      }
    }

**Second Example**

In [28]:
messages.clear()

In [29]:
base_system_message = "You are a helpful assistant that converts natural language into SQL."

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant that converts natural language into SQL.


In [30]:
# This is the first user message that will be sent to the model. Feel free to update this.
user_message = f"""Write sql code which merges 2 tables and finds the maximum value of salary column"""

In [31]:
# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [32]:
# A sample API call for chat completions looks as follows:
# Messages must be an array of message objects, where each object has a role (either "system", "user", or "assistant") and content (the content of the message).
# For more info: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#chat-completions
# You can get "resource not found error in case your version isn't correct"

model_name= "gpt-4-32k"

try:
    max_response_tokens = 500

    response = send_message(messages, model_name, max_response_tokens)
    messages.append({"role": "assistant", "content": response})

    print_conversation(messages)    
   
except openai.error.APIError as e:
    # Handle API error here, e.g. retry or log
    print(f"OpenAI API returned an API Error: {e}")

except openai.error.AuthenticationError as e:
    # Handle Authentication error here, e.g. invalid API key
    print(f"OpenAI API returned an Authentication Error: {e}")

except openai.error.APIConnectionError as e:
    # Handle connection error here
    print(f"Failed to connect to OpenAI API: {e}")

except openai.error.InvalidRequestError as e:
    # Handle connection error here
    print(f"Invalid Request Error: {e}")

except openai.error.RateLimitError as e:
    # Handle rate limit error
    print(f"OpenAI API request exceeded rate limit: {e}")

except openai.error.ServiceUnavailableError as e:
    # Handle Service Unavailable error
    print(f"Service Unavailable: {e}")

except openai.error.Timeout as e:
    # Handle request timeout
    print(f"Request timed out: {e}")
    
except:
    # Handles all other exceptions
    print("An exception has occured.")

[SYSTEM]
You are a helpful assistant that converts natural language into SQL.

[USER]
Write sql code which merges 2 tables and finds the maximum value of salary column

[ASSISTANT]
{
  "id": "chatcmpl-7oAsQ4VWSXqbbO0zKVGCgqpGQz558",
  "object": "chat.completion",
  "created": 1692192566,
  "model": "gpt-4-32k",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "To merge two tables and find the maximum value of a salary column, you would need to use a UNION clause to combine the tables and then a MAX function to find the maximum salary. However, the exact SQL query will depend on the structure of your tables. Here's a general example:\n\n```sql\nSELECT MAX(salary) \nFROM (\n    SELECT salary FROM table1\n    UNION ALL\n    SELECT salary FROM table2\n) AS combined_table\n```\n\nReplace `table1` and `table2` with the names of your tables. This query assumes that both tables have a column called `salary`